In [12]:
from pathlib import Path
import pandas as pd

artifacts = Path("/media/cumulus/curation_data/modality_classifiers_production/models/cord19/")
data_path = Path("/media/cumulus/curation_data/modality_classifiers_production/data/cord19")

classifiers = {
    "classifier": "higher-modality",
    "classname": "",
    "path": artifacts / "higher-modality/efficientnet-b1_higher-modality_0.pt",
    "children": [
        {
            "classifier": "experimental",
            "classname": "exp",
            "path": artifacts / "experimental/efficientnet-b1_experimental_0.pt",
            "children": [
                {
                    "classifier": "gel",
                    "classname": "exp.gel",
                    "path": artifacts / "gel/efficientnet-b1_gel_0.pt",
                    "children": []
                }
            ],
        },
        {
            "classifier": "graphics",
            "classname": "gra",
            "path": artifacts / "graphics/efficientnet-b1_graphics_0.pt",
            "children": [],
        },
        {
            "classifier": "microscopy",
            "classname": "mic",
            "path": artifacts / "microscopy/efficientnet-b0_microscopy_0.pt",
            "children": [
                {
                    "classifier": "electron",
                    "classname": "mic.ele",
                    "path": artifacts / "electron/efficientnet-b1_electron_0.pt",
                    "children": [],
                },
            ],
        },
        {
            "classifier": "molecular",
            "classname": "mol",
            "path": artifacts / "molecular" / "efficientnet-b1_molecular_0.pt",
            "children": [],
        },
        {
            "classifier": "radiology",
            "classname": "rad",
            "path": artifacts / "radiology" / "efficientnet-b0_radiology_0.pt",
            "children": [],
        },
        {
            "classifier": "photography",
            "classname": "pho",
            "path": artifacts / "photography/resnet34_photography_0.pt",
            "children": []
        }
    ],
}



model_path = data_path / "cord19_microscopy_v1.parquet"
df = pd.read_parquet(model_path)
# df = df[:10]

In [13]:
from image_modalities_classifier.models.predict import ModalityPredictor, RunConfig
from os import cpu_count

config = RunConfig(32, cpu_count(), "cuda:0")
predictor = ModalityPredictor(classifiers, config)

In [14]:
base_img_path = "/home/jtt/Documents/datasets/curation_data/"
img_paths = df.img_path.values.tolist()
predictions = predictor.predict(img_paths, base_img_path)

/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.wa

In [15]:
predictions[:100]

,img_path,prediction
0,subfigure-classification/2013/train/DMFL/1471-...,mic.flu
1,subfigure-classification/2013/train/DMFL/1471-...,mic.flu
2,subfigure-classification/2013/train/DMFL/1471-...,mic.flu
3,subfigure-classification/2013/train/DMFL/1471-...,mic.flu
4,subfigure-classification/2013/train/DMFL/1471-...,mic.flu
...,...,...
95,subfigure-classification/2013/train/DMLI/cde00...,mic.lig
96,subfigure-classification/2013/train/DMLI/DRP20...,mic.lig
97,subfigure-classification/2013/train/DMLI/IJD-5...,mic.lig
98,subfigure-classification/2013/train/DMLI/IJD-5...,mic.lig


In [16]:
predictions.prediction.unique()

array(['mic.flu', 'oth', 'mic.lig', 'mic.ele.tra', 'mic.ele.sca',
       'exp.pla', 'pho.org', 'mol.dna', 'mol.3ds', 'exp.gel.wes',
       'exp.gel.rpc', 'exp.gel.nor', 'pho.der', 'rad.ang'], dtype=object)